In [1]:
import requests
from bs4 import BeautifulSoup
import re
import itertools
import pandas as pd, numpy as np
import os
import glob
import pathlib
import re
import janome
import jaconv
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *

In [2]:
url = "https://bunshun.jp/"
res = requests.get(url)
soup = BeautifulSoup(res.text, "html.parser")
elems = soup.select("ul")

In [3]:
title_list = []
url_list = []
for sibling in elems[3]:
    if sibling != "\n":
        print(sibling.h3.string)
        title_list.append(sibling.h3.string.replace('\u3000', ' '))
        print(url + sibling.h3.a["href"])
        url_list.append(url + sibling.h3.a["href"])

ジャニーズ元MADEリーダー・稲葉光（29）が元Berryz工房アイドルと“渋谷ホテルデート”《スクープ撮》
https://bunshun.jp//articles/-/40869
「駅近、約1,200㎡の広大な敷地」「自宅の建て替えと地域貢献を両立する土地活用」プロが出した答えとは？
https://bunshun.jp//articles/-/40010
「365日24時間働こう」……ワタミの“思想教育”はいまも続いていた
https://bunshun.jp//articles/-/40843
“縦割り行政”の弊害だ！ 橋下徹が語る「Go Toキャンペーンは何が間違っているのか」
https://bunshun.jp//articles/-/40877
警察官に唾吐き公務執行妨害で逮捕　読売新聞がソウル支局記者を懲戒処分
https://bunshun.jp//articles/-/40868
同性愛差別の足立区議宛てに「とんでもない思い違いです」…81歳祖母の手紙が教えてくれたこと
https://bunshun.jp//articles/-/40826
家系ラーメン“のれん分け戦争”「吉村家vs.六角家」裏切りと屈服の黒歴史〈六角家破産〉
https://bunshun.jp//articles/-/40752
《平手派また卒業》欅坂46・佐藤詩織が書いた「活動のなかで悲しかったこと」　櫻坂46の“急勾配”を運営も懸念
https://bunshun.jp//articles/-/40862
安倍政権最大の功績は“アイヌ博物館”だった？　200億円をブチ込んだ「ウポポイ」の虚実
https://bunshun.jp//articles/-/40841
「長さ」ではなく…美容師が教える、髪を切るときに言うといい「意外な言葉」
https://bunshun.jp//articles/-/40694


In [4]:
pickup_links = [url + sibling.h3.a["href"] for sibling in elems[3] if sibling != "\n"]

In [5]:
news_list = []
for pickup_link in pickup_links:
    news = []
    news.append(pickup_link)
    pickup_res = requests.get(pickup_link)
    pickup_soup = BeautifulSoup(pickup_res.text, "html.parser")
    while True:
        try:
            next_link = pickup_soup.find("a", class_="next menu-link ga_tracking")["href"]
            next_link = url + next_link
            next_res = requests.get(next_link)
            pickup_soup = BeautifulSoup(next_res.text, "html.parser")
            news.append(next_link)
        except Exception:
            news_list.append(news)
            break

In [6]:
news_page_list = []
for news_links in news_list:
    news_page = ''
    for news_link in news_links:
        news_res = requests.get(news_link)
        news_soup = BeautifulSoup(news_res.text, "html.parser")
        news_soup = news_soup.find(class_=re.compile("article-body")).find_all("p")
        news_phrase = ''
        for news in news_soup:
            news_phrase += news.text.replace('\u3000', ' ')
        news_page += news_phrase
    news_page_list.append(news_page)

In [7]:
new_no_list = [x for x in range(len(title_list))]

In [8]:
newslist = np.array([new_no_list, title_list, url_list, news_page_list]).T
newslist = pd.DataFrame(newslist, columns=['ニュースNo.', 'title', 'url', 'news_page_list'])
newslist = newslist.astype({'ニュースNo.':'int64'})

In [9]:
newslist

,ニュースNo.,title,url,news_page_list
0,0,ジャニーズ元MADEリーダー・稲葉光（29）が元Berryz工房アイドルと“渋谷ホテルデート...,https://bunshun.jp//articles/-/40869,ジャニーズJr.内の人気ユニット「宇宙Six」の山本亮太（30）が違法な闇スロット店に通っ...
1,1,"「駅近、約1,200㎡の広大な敷地」「自宅の建て替えと地域貢献を両立する土地活用」プロが出し...",https://bunshun.jp//articles/-/40010,多くの土地オーナーから「土地活用のパートナー」として選ばれ続けている三井ホーム。土地活用のプ...
2,2,「365日24時間働こう」……ワタミの“思想教育”はいまも続いていた,https://bunshun.jp//articles/-/40843,ワタミ株式会社の労働問題に関する告発が続いている。10月2日、「ワタミの宅食」営業所の所長...
3,3,“縦割り行政”の弊害だ！ 橋下徹が語る「Go Toキャンペーンは何が間違っているのか」,https://bunshun.jp//articles/-/40877,「規制改革」「行政改革」「縦割り打破」。 菅義偉政権発足後、「改革」という言葉がよく聞こえて...
4,4,警察官に唾吐き公務執行妨害で逮捕 読売新聞がソウル支局記者を懲戒処分,https://bunshun.jp//articles/-/40868,読売新聞ソウル支局の記者（34）が7月中旬、公務執行妨害の容疑で韓国当局に逮捕されていたこ...
5,5,同性愛差別の足立区議宛てに「とんでもない思い違いです」…81歳祖母の手紙が教えてくれたこと,https://bunshun.jp//articles/-/40826,「おばあちゃんが足立区議の件で怒っていて、手紙を書くらしい」 母からこんなLINEが来たの...
6,6,家系ラーメン“のれん分け戦争”「吉村家vs.六角家」裏切りと屈服の黒歴史〈六角家破産〉,https://bunshun.jp//articles/-/40752,「『六角家』は破産しましたが、ジャンルとしての『家系ラーメン』は、年々店舗数が拡大しており、...
7,7,《平手派また卒業》欅坂46・佐藤詩織が書いた「活動のなかで悲しかったこと」 櫻坂46の“急勾...,https://bunshun.jp//articles/-/40862,《皆さんこんばんは。今日はいつも応援して頂いている皆様にお伝えしたいことがあります。私、佐藤...
8,8,安倍政権最大の功績は“アイヌ博物館”だった？ 200億円をブチ込んだ「ウポポイ」の虚実,https://bunshun.jp//articles/-/40841,──本当にあれでいいんだろうか？ 帰路、雨の道央道をレンタカーでひた走りながら、そんな思いが...
9,9,「長さ」ではなく…美容師が教える、髪を切るときに言うといい「意外な言葉」,https://bunshun.jp//articles/-/40694,掛け布団がだんだん心地よくなってきた、今日この頃。朝夜の急激な気温差に、薄めの上着を羽織っ...


In [10]:
newslist['ニュースNo.'].dtype

dtype('int64')

In [11]:
p_dic = pathlib.Path('/work/dic')

for i in p_dic.glob('*.txt'):
    with open (i, 'r', encoding='utf-8') as f:
        x = [i.replace('\n', '').split(':') for i in f.readlines()]
        
posi_nega_df = pd.DataFrame(x, columns = ['基本形', '読み', '品詞', 'スコア'])
posi_nega_df['読み'] = posi_nega_df['読み'].apply(lambda x : jaconv.hira2kata(x))
posi_nega_df = posi_nega_df[~posi_nega_df[['基本形', '読み', '品詞']].duplicated()]

In [12]:
i = 0

t = Tokenizer()
char_filters = [UnicodeNormalizeCharFilter()]
analyzer = Analyzer(char_filters=char_filters, tokenizer=t)

word_lists = []
for i, row in newslist.iterrows():
    for t in analyzer.analyze(row[3]):
        surf = t.surface
        base = t.base_form
        pos = t.part_of_speech
        reading = t.reading
        
        word_lists.append([i, surf, base, pos, reading])
word_df = pd.DataFrame(word_lists, columns=['ニュースNo.', '単語', '基本形', '品詞', '読み'])
word_df['品詞'] = word_df['品詞'].apply(lambda x : x.split(',')[0])

In [13]:
score_result = pd.merge(word_df, posi_nega_df, on=['基本形', '品詞', '読み'], how='left')

In [14]:
result = []
for i in range(len(score_result['ニュースNo.'].unique())):
    temp_df = score_result[score_result['ニュースNo.']== i]
    text = ''.join(list(temp_df['単語']))
    score = temp_df['スコア'].astype(float).sum()
    score_r = score/temp_df['スコア'].astype(float).count()
    result.append([i, text, score, score_r])

ranking = pd.DataFrame(result, columns=['ニュースNo.', 'テキスト', '累計スコア', '標準化スコア']).sort_values(by='標準化スコア').reset_index(drop=True)

In [15]:
newslist[['ニュースNo.', 'title', 'url']]['ニュースNo.'].dtype

dtype('int64')

In [16]:
ranking['ニュースNo.'].dtype

dtype('int64')

In [17]:
ranking = pd.merge(ranking, newslist[['ニュースNo.', 'title', 'url']], on='ニュースNo.', how='left')
ranking = ranking.reindex(columns=['ニュースNo.', 'title', 'url', 'テキスト', '累計スコア', '標準化スコア'])

In [18]:
print("<<ポジティブ1位>>", end="\n\n")
print(ranking.iloc[-1, 1])
print(ranking.iloc[-1, 2])
print(ranking.iloc[-1, 3])

<<ポジティブ1位>>

家系ラーメン“のれん分け戦争”「吉村家vs.六角家」裏切りと屈服の黒歴史〈六角家破産〉
https://bunshun.jp//articles/-/40752
「『六角家』は破産しましたが、ジャンルとしての『家系ラーメン』は、年々店舗数が拡大しており、今が最盛期です。その背景には“資本系”の台頭があります。でもね、確かに資本系も美味しいんですが、かつて『六角家』や『吉村家』の職人たちが日々、味の改良に挑み、切磋琢磨して、互いに覇を競っていたあの時代が、倒産のニュースを聞いた今、妙に懐かしく輝いて見えるんです」(ラーメン評論家・山本剛志氏)(前編「家系ラーメン・名門『六角家』はなぜ破産したか」を読む) 2020年9月4日、かつて家系ラーメンの「代名詞」とまで呼ばれ、一世を風靡した老舗の名店「六角家」が、横浜地裁より破産手続き開始決定を受けていると報じられた。2017年に「六角家」の本店が閉じられ、倒産は時間の問題だったとみるファンも多かったが、いざこのニュースが流れると「マジか」「もう一度食べたかった」など、その倒産を惜しむ声がネット上などで多く上がった。そして、冒頭の山本氏のように、「六角家」がこれまで「家系ラーメン」業界の隆盛にはたした役割について、再評価する声も上がってきている。「そもそも、家系ラーメンが誕生したのは1974年。長距離トラックの運転手だった吉村実氏が仕事の合間に趣味で密かに研究を重ねていた九州の豚骨ベースと東京の醤油ベースを組み合わせたスープを開発したことに端を発しています。この味ならいける! と判断した吉村氏はその後会社を辞め、『吉村家』というラーメン店を横浜の磯子区新杉田に開店。豚骨醤油ベースの風味豊かなスープと、モッチリとした『酒井製麺』の太麺を合わせた1杯は、本人の読みどおり大きな人気を集め、店は日々、大勢のお客さんで賑わいました。 評判を集めた「吉村家」には、多くの才能が集まりました。そして、吉村氏の元で修業をした弟子たちは、吉村氏直々に教わり技術を身に着けると“暖簾分け”という形で自分の店を持つようになり、弟子たちの店もそれぞれのエリアで大人気店となりました。独立した弟子たちが店名に『~家』とつけることが多かったため、ファンの間で、『吉村家』とそこから独立した店は『家系』と呼ばれるようになりました。現在は、

In [19]:
print("<<ネガティブ1位>>", end="\n\n")
print(ranking.iloc[0, 1])
print(ranking.iloc[0, 2])
print(ranking.iloc[0, 3])

<<ネガティブ1位>>

「365日24時間働こう」……ワタミの“思想教育”はいまも続いていた
https://bunshun.jp//articles/-/40843
ワタミ株式会社の労働問題に関する告発が続いている。10月2日、「ワタミの宅食」営業所の所長が、労働基準監督署からの残業代未払いの是正勧告、月175時間を超える長時間労働、上司によるタイムカードの改ざんを次々と公表したのだ。「ホワイト企業」宣伝のワタミで月175時間の残業 残業代未払いで労基署から是正勧告ワタミがホワイト企業になれなかった理由は? 勝手に勤怠「改ざん」システムも Aさんは長時間労働の末、昼夜の感覚がなくなり、「このまま寝たら、もう目が覚めないのではないか」と恐怖を抱きながら生活するほどだった。「あのまま働いていたら、死んでいた」とAさんは断言する。現在は、精神疾患を発症し、労災申請をしながら休業中だ。 しかし、Aさんは命の危険を感じていながら、なぜ過酷な仕事を続けてしまったのだろうか。その背景には、労働者の意識に働きかけ、過酷な労働を受け入れさせてしまう、ワタミによる「思想教育」のシステムがあった。「こんなにいい仕事をしているんだから、苦しくても頑張ろう」「苦しいことも、苦しくない。むしろ自分の力になる」 Aさんは過重労働の最中、そう自分に言い聞かせていた。 実際、Aさんはワタミの宅食の仕事に「誇り」を感じていたという。確かにワタミの「宅食」に「社会貢献」と言える部分がある。ワタミの宅食には、食事の調達が難しい高齢者に、定期的に安く食事を届けるというコンセプトがある。Aさんの営業所の届け先も、そうした人たちばかりだった。 隔日でデイサービスに通い、隔日で家にいるときの食事に困る一人暮らしの高齢者。毎日昼食用と夕食用の2食を頼んで、ワタミの宅食しか食べていない高齢者、障がい者や、親を介護している人......。 苦しい生活の事情によって、食事に手間や時間、お金をかけられず、ワタミに頼らざるを得ない様々な利用者がいた。それまで介護や教育現場で働いてきたAさんにとって、社会や地域のためになる宅食の仕事は、非常にやりがいの感じられるものだった。 だがそれらの「支援」を通じて無理やり利益をあげようとすれば、長時間労働や低賃金による労働者の犠牲によって支えられることになってしまう。いわば「貧困・ブ